# Test PointPillars on ARCS data

In [275]:
# Libraries
import bbox
import itertools
import math
import os
import random
import sys
import time
import numpy as np
import pandas as pd
from contextlib import redirect_stdout
from mmdet3d.apis import LidarDet3DInferencer
from pathlib import Path

In [276]:
# Initialize inferencer
inferencer = LidarDet3DInferencer('pointpillars_kitti-3class')

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection3d/v1.0.0_models/pointpillars/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth


/home/rachel/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(
/home/rachel/miniconda3/envs/openmm_mmvc/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [277]:
# Directory paths
ARCS_LABELS = '../data/eval_data/labels'
ARCS_DATA_DIR = '../data/eval_data/arcs'
ARCS_DISTANCE_FILTERED_DATA_DIR = '../data/eval_data/arcs_distance_filtered'
ARCS_HEIGHT_FILTERED_DATA_DIR = '../data/eval_data/arcs_height_filtered'
ARCS_LABEL_FILTERED_DATA_DIR = '../data/eval_data/arcs_label_filtered'

In [278]:
# Directory paths
# ARCS_LABELS = '../data/eval_data/labels'
# ARCS_DATA_DIR = '../data/pcap_filter_data/arcs_pcap_unfiltered'
# PREV_ARCS_AZIMUTH_FILTERED_DATA_DIR = '../data/pcap_filter_data/arcs_pcap_filter'
# ARCS_AZIMUTH_FILTERED_DATA_DIR = '../data/eval_data/arcs_azimuth_filtered'
# ARCS_LABEL_FILTERED_DATA_DIR = '../data/eval_data/arcs_label_filtered'

## Evaluation functions

In [279]:
# The 3D bbox object requires quaternion values. This function converts the yaw to these values
def yaw_to_quaternion(yaw):
    """
    Converts a yaw angle (rotation about the z-axis) to quaternion coordinates.
    
    Parameters:
    - yaw (float): The yaw angle in radians.
    
    Returns:
    - tuple of (rw, rx, ry, rz): The quaternion representation of the yaw.
    """
    # Compute the components of the quaternion
    rw = math.cos(yaw / 2.0)
    rz = math.sin(yaw / 2.0)
    
    # rx and ry are zero because the rotation is only about the z-axis
    return (rw, 0, 0, rz)

In [280]:
# This function takes x, y, z, dx, dy, dz, yaw and returns a 3D bbox object from the bbox package
# Yaw is in radians
def get3DBbox(x, y, z, dx, dy, dz, yaw):
    # Example usage:
    rw, rx, ry, rz = yaw_to_quaternion(yaw)
    bbox_obj = bbox.BBox3D(x, y, z, length=dx, width=dy, height=dz, rw=rw, rx=rx, ry=ry, rz=rz)
    return bbox_obj

In [281]:
# Takes two bounding boxes, returns the IoU
def calculate_iou(bbox1, bbox2):
    bbox_obj1 = get3DBbox(*bbox1)
    bbox_obj2 = get3DBbox(*bbox2)
    return bbox.metrics.jaccard_index_3d(bbox_obj1, bbox_obj2)

In [282]:
evaluation_area = [21.0175, 11.717, -0.3925, 40.035, 55.9349, 7.535, 0.5]

In [283]:
# Returns true if the pending box overlaps the evaluation area
def is_in_eval_area(pending_bbox):
    iou = calculate_iou(evaluation_area, pending_bbox)
    if iou > 0:
        return True
    return False
#     return True

In [284]:
# _evaluate frame takes a prediction dictionary output by the model, and a list of ground truths
# from the label file, and returns the TPs, FPs, and FNs for one lidar frame
def _evaluate_frame(predictions, ground_truths, iou_threshold=0.25):
    TPs, FPs, FNs = 0, 0, len(ground_truths)
    used_gt = set()
    confidence_labels = []
    
    for pred in predictions['predictions']:
        for label, score, bbox in zip(pred['labels_3d'], pred['scores_3d'], pred['bboxes_3d']):
            # Only count the prediction if it's within the evaluation area
            if is_in_eval_area(bbox):
                best_iou = 0
                best_gt = None
                for gt in ground_truths:
                    iou = calculate_iou(bbox, gt[8:])
                    if iou > best_iou:
                        best_iou = iou
                        best_gt = tuple(gt)

                if best_iou > iou_threshold:
                    if best_gt not in used_gt:
                        used_gt.add(best_gt)
                        TPs += 1
                        FNs -= 1
                        confidence_labels.append((score, 1))
                else:
                    FPs += 1
                    confidence_labels.append((score, 0))

    return TPs, FPs, FNs, confidence_labels

In [285]:
def parse_ground_truths(label_path):
    labels = []
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            bbox = []
            # Add the category as a string
            bbox.append(parts[0])
            # Extract the bounding box dimensions and location as 
            bbox = bbox + [float(value) for value in parts[1:15]]  
            bbox[2] = int(bbox[2])
            # Only include the bbox if it's in the evaluation area
            if is_in_eval_area(bbox[8:]):
                labels.append(bbox)
    return labels

In [286]:
generated_problem_file_list = []

In [287]:
# _evaluate frame takes a lidar and label file path and
# and returns the TPs, FPs, and FNs for one lidar frame
def evaluate_frame(lidar_file_path, label_file_path):
    inputs = dict(points=str(lidar_file_path))
    
    # Get predictions
    try:
        predictions = inferencer(inputs)
            # Get ground truths
        ground_truths = parse_ground_truths(label_file_path)
        num_ground_truths = len(ground_truths)

        TPs, FPs, FNs, confidence_labels = _evaluate_frame(predictions, ground_truths)
    except:
        # Add file id to the problem file list
        print(str(lidar_file_path))
        lidar_filename = os.path.basename(lidar_file_path)
        file_id, extension = os.path.splitext(lidar_filename)
        generated_problem_file_list.append(file_id)
        TPs, FPs, FNs, num_ground_truths, confidence_labels = 0, 0, 0, 0, []
    
    return TPs, FPs, FNs, num_ground_truths, confidence_labels

In [288]:
# Problem files. I'll find out why later
problem_file_ids = ['006437', '005158', '001010', '004928', '000522', '000444', '000863', '004251', 
                    '000523', '000443', '000530', '000520', '006436', '004344', '000535', '004223', 
                    '001016', '000552', '004287', '004226', '004289', '001032', '000767', '003395', 
                    '005319', '000554', '004227', '004012', '000445', '000524', '001023', '005317', 
                    '004346', '000548', '000558', '001026', '004345', '000537', '000531', '000526', 
                    '000549', '001022', '004926', '000664', '003438', '004229', '000553', '001027', 
                    '000532', '003399', '003066', '004930', '000663', '004929', '000529', '005322', 
                    '001013', '000559', '005155', '004252', '003436', '004291', '001011', '000866', 
                    '000662', '004016', '004224', '000556', '001015', '000543', '001020', '001024', 
                    '000525', '004225', '001029', '000533', '000527', '001028', '006439', '001017', 
                    '003437']

In [289]:
def print_summary(list_file_ids):
    for id in list_file_ids:
        print(f'\n_____________________________________________________________')
        
        file_path = f'{ARCS_LABEL_FILTERED_DATA_DIR}/{id}.bin'
        label_path = f'{ARCS_LABELS}/{id}.txt' 

        # Open point cloud file
        points = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
        df_points = pd.DataFrame(points, columns=['x', 'y', 'z', 'intensity'])

        # Print the DataFrame summary
        print(f"Summary for file ID {id}:")
        print('length: ' + str(len(df_points)))
        print("DataFrame Info:")
        df_points.info()
        print("\nDataFrame Description:")
        print(df_points.describe())
        print(f'\nlabels:')
        # Print labels
        with open(label_path, 'r') as file:
            for line in file:
                print(line)

## Metrics calculation functions

In [290]:
def calculate_precision_recall(predictions, num_ground_truths):
    # Sort by confidence score in descending order
    predictions.sort(key=lambda x: x[0], reverse=True)
    
    tp = 0
    fp = 0
    total_positives = sum(is_tp for _, is_tp in predictions)
    
    precisions = []
    recalls = []
    
    for conf, is_tp in predictions:
        if is_tp:
            tp += 1
        else:
            fp += 1
        
        precision = tp / (tp + fp)
        recall = tp / num_ground_truths
        
        precisions.append(precision)
        recalls.append(recall)
    
    return precisions, recalls

In [291]:
def calculate_ap(precisions, recalls):
    # Calculate AP using the trapezoidal rule to compute the area under the curve
    ap = 0
    for i in range(1, len(recalls)):
        ap += (recalls[i] - recalls[i-1]) * (precisions[i] + precisions[i-1]) / 2
    return ap

In [292]:
test_dir = Path(ARCS_DATA_DIR)
files = [f for f in os.listdir(test_dir) if f.endswith('.bin')]
random.shuffle(files)
files = files[:1000]
files = [file for file in files if file[:-4] not in problem_file_ids]
# print(files)

In [293]:
def evaluate_dataset(dataset_path):
    label_dir = Path(ARCS_LABELS)
    dataset_dir = Path(dataset_path)
    
    total_TPs, total_FPs, total_FNs = 0, 0, 0
    num_ground_truths = 0
    confidence_labels = []

    start = time.time()
    for bin_file in files:
#     for bin_file in itertools.islice(dataset_dir.iterdir(), num_frames):
#     for bin_file in dataset_dir.iterdir():
        if str(bin_file).endswith('.bin'):
            print('.', end='')

            lidar_filename = os.path.basename(bin_file)
            # Split the filename from the extension ('006428', '.txt')
            file_id, extension = os.path.splitext(lidar_filename)
            if file_id not in generated_problem_file_list and file_id not in problem_file_ids:
                label_filename = file_id + '.txt'

                # Make file paths
                lidar_file_path = Path(dataset_dir, lidar_filename)
                label_file_path = Path(label_dir, label_filename)
                
                
                TPs, FPs, FNs, num_frame_ground_truths, frame_confidence_labels = evaluate_frame(lidar_file_path, label_file_path)

                total_TPs += TPs
                total_FPs += FPs
                total_FNs += FNs
                num_ground_truths += num_frame_ground_truths
                confidence_labels += frame_confidence_labels
            else:
                print('skipping ' + file_id)
    end = time.time()
        
    return total_TPs, total_FPs, total_FNs, num_ground_truths, confidence_labels, (end - start) / len(files)

In [294]:
def test_pointpillars(dataset_path, dataset_name):
    print('Evaluating dataset: ' + dataset_path)
    # Get all TP, FP, and FN in the dataset
    TPs, FPs, FNs, num_ground_truths, confidence_labels, time_per_frame = evaluate_dataset(dataset_path)
    # Get metrics
    precision = TPs / (TPs + FPs) if TPs + FPs > 0 else 0
    recall = TPs / (TPs + FNs) if TPs + FNs > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Placeholder for average precision calculation
    precisions, recalls = calculate_precision_recall(confidence_labels, num_ground_truths)
    print(confidence_labels)
    print(precisions)
    print(recalls)
    average_precision = calculate_ap(precisions, recalls)

    # Organize results into a dictionary
    results = {
        'Dataset': dataset_name,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1_score,
        'Average Precision': average_precision,
        'time_per_frame': time_per_frame
    }
    print(results)

    return results

In [295]:
%%capture
# Run tests on ARCS filtered - this goes first if there's a problem file, it doesn't get evaluated for other sets
height_filter_results = test_pointpillars(ARCS_HEIGHT_FILTERED_DATA_DIR, 'Height Filtered ARCS')

In [296]:
%%capture
# Run tests on ARCS
results = test_pointpillars(ARCS_DATA_DIR, 'ARCS')

In [297]:
%%capture
# Run tests on ARCS filtered
distance_filter_results = test_pointpillars(ARCS_DISTANCE_FILTERED_DATA_DIR, 'Distance Filtered ARCS')

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [298]:
%%capture
# Run tests on ARCS label filtered
label_filter_results = test_pointpillars(ARCS_LABEL_FILTERED_DATA_DIR, 'Label Filtered ARCS')

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [299]:
print(generated_problem_file_list)

['006441', '004347', '003065', '000546', '001008', '006438', '000534', '005318', '004054', '000542', '004011', '001031', '000521']


## Show results

In [300]:
# print(results)
# print(filter_results)
# print(label_filter_results)

In [301]:
# Create DataFrame
results_df = pd.DataFrame([results, distance_filter_results, height_filter_results, label_filter_results])
# results_df = pd.DataFrame([results, azimuth_filter_results])
display(results_df)

,Dataset,Precision,Recall,F1 Score,Average Precision,time_per_frame
0,ARCS,0.567485,0.729623,0.638420,0.635596,0.152910
1,Distance Filtered ARCS,0.418706,0.802366,0.550263,0.668195,0.130892
2,Height Filtered ARCS,0.592723,0.813760,0.685873,0.697300,0.115643
3,Label Filtered ARCS,0.809587,0.809231,0.809409,0.784855,0.107207


In [302]:
print(results_df.to_string(index=False))

               Dataset  Precision   Recall  F1 Score  Average Precision  time_per_frame
                  ARCS   0.567485 0.729623  0.638420           0.635596        0.152910
Distance Filtered ARCS   0.418706 0.802366  0.550263           0.668195        0.130892
  Height Filtered ARCS   0.592723 0.813760  0.685873           0.697300        0.115643
   Label Filtered ARCS   0.809587 0.809231  0.809409           0.784855        0.107207
